# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [1]:
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

url = RESOURCE_ENDPOINT + "openai/deployments/" + DEPLOYMENT_ID + "/chat/completions?api-version=" + API_VERSION

print(url)

Found Azure OpenAI API Base Endpoint: https://ahmedbham-aoai.openai.azure.com/
https://ahmedbham-aoai.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview


In [2]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = requests.post(url, headers={"api-key": API_KEY}, json={"messages":messages})
    response_json = response.json()
    return response_json["choices"][0]["message"]["content"]
    # return response

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing the latest addition to our mid-century inspired office furniture collection - the SWC Chair. With its sleek design and available in several options of shell color and base finishes, this chair is sure to complement any home or business setting. The SWC Chair is suitable for contract use and a qualified addition to any professional workspace.

Constructed with a 5-wheel plastic-coated aluminum base and pneumatic chair adjust for easy raise/lower action, the SWC Chair is designed with comfort in mind. It has the option of plastic back and front upholstery or full upholstery in 10 fabric and 6 leather options. Choose between soft or hard-floor caster options, and two choices of seat foam densities: medium or high.

The SWC Chair has four base finish options: stainless steel, matte black, gloss white, or chrome. It also has the option of armless or 8 position PU armrests. The chair measures: 53 cm (20.87") width, 51 cm (20.08") depth, 80 cm (31.50") height, 44 cm (17.32") seat 

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Available in various colors and finishes, with options for upholstery and armrests. Designed for both home and business use, with a durable construction and adjustable height feature. Made in Italy for quality assurance.


In [ ]:
len(response.split())

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [10]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Choose from a variety of shell colors and base finishes to suit your space. Constructed with durable materials like cast aluminum and HD36 foam, this chair is perfect for both home and business use. Made in Italy.


In [11]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile mid-century inspired office chair, available in a range of shell colors and base finishes. Constructed with a durable cast aluminum shell and a comfortable HD36 foam seat. Choose from various upholstery options and armrest configurations to suit your needs. Perfect for both home and office use. 

Product IDs: SWC-100, SWC-110


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [12]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <p>This mid-century inspired office chair is a perfect addition to any home or business setting. With a variety of shell colors and base finishes to choose from, you can customize this chair to fit your style. The chair is available with plastic back and front upholstery or full upholstery in a range of fabric and leather options. The 5-wheel plastic coated aluminum base provides stability, while the pneumatic chair adjust allows for easy height adjustment. Made with high-quality materials such as cast aluminum with modified nylon coating and HD36 foam, this chair is not only stylish but also durable. Choose between soft or hard-floor casters and two seat foam densities to create the perfect seating experience. Whether you prefer armless or PU armrests, this chair offers comfort and style in one package. Qualified for contract use, this chair is a versatile and practical choice for any space.</p>
  
  <p>Product IDs: SWC-100, SWC-110</p>
  
  <h2>Product Dimensions</h2>
  <tabl

## Load Python libraries to view HTML

In [13]:
from IPython.display import display, HTML

In [14]:
display(HTML(response))

## Try experimenting on your own!